In [1]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit
import json
import anyio
import asyncio

# Only needed for Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

from autogen.agentchat.group import (
    AgentNameTarget,
    AgentTarget,
    AskUserTarget,
    ContextExpression,
    ContextStr,
    ContextStrLLMCondition,
    ContextVariables,
    ExpressionAvailableCondition,
    ExpressionContextCondition,
    GroupChatConfig,
    GroupChatTarget,
    Handoffs,
    NestedChatTarget,
    OnCondition,
    OnContextCondition,
    ReplyResult,
    RevertToUserTarget,
    SpeakerSelectionResult,
    StayTarget,
    StringAvailableCondition,
    StringContextCondition,
    StringLLMCondition,
    TerminateTarget,
)

from autogen.agentchat.group.patterns import (
    DefaultPattern,
    ManualPattern,
    AutoPattern,
    RandomPattern,
    RoundRobinPattern,
)


from autogen import ConversableAgent, UpdateSystemMessage
from autogen.agents.experimental import DocAgent
import os
import copy
from typing import Any, Dict, List
from pydantic import BaseModel, Field


from autogen.agentchat import initiate_group_chat, a_initiate_group_chat

In [2]:
# Path to the arxiv MCP server
mcp_server_path = Path("mcp_notion.py")

In [3]:
from google_auth_oauthlib.flow import InstalledAppFlow
import json

In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

import os

# Access tokens
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
from mcp_notion import NotionReader

In [6]:
# Set up the Notion MCP server
notion_server_path = Path("mcp_notion.py")

In [7]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client


In [8]:
PAGE_ID = "160ab8e5212680239210c6501484cd90"

In [9]:
async def read_notion_page():
    """Read content from a specific Notion page using the NotionReader."""
    try:
        # Create a NotionReader instance
        reader = NotionReader()
        
        print(f"Reading Notion page with ID: {PAGE_ID}")
        
        # Use the reader as a context manager to ensure proper cleanup
        async with reader:
            # Get page content
            page = await reader.get_page(PAGE_ID)
            
            # Get page blocks
            blocks_response = await reader.get_block_children(PAGE_ID)
            blocks = blocks_response.get("results", [])
            
            # Format the results for easier consumption
            formatted_result = format_page_content(page, blocks)
            
            # Print a summary of what we found
            print(f"Page title: {formatted_result.get('title', 'Untitled')}")
            print(f"Found {len(formatted_result.get('content', []))} content blocks")
            
            # Save the full content to a JSON file for inspection
            with open("notion_page_content.json", "w") as f:
                json.dump(formatted_result, f, indent=2)
            print("Full content saved to notion_page_content.json")
            
            # Also print the first few blocks
            print("\nPreview of content:")
            for i, block in enumerate(formatted_result.get('content', [])[:5]):  # First 5 blocks
                block_text = block.get('text', '')[:100]  # First 100 chars
                print(f"{i+1}. [{block.get('type')}]: {block_text}..." if len(block_text) > 100 else block_text)
            
            return formatted_result
            
    except Exception as e:
        print(f"Error reading Notion page: {str(e)}")
        raise


In [10]:
def format_page_content(page, blocks):
    """Format page and blocks into a more user-friendly structure."""
    # Extract title if available
    title = ""
    for prop_name, prop_value in page.get("properties", {}).items():
        if prop_value.get("type") == "title" and prop_value.get("title"):
            title = "".join([text.get("plain_text", "") for text in prop_value.get("title", [])])
            break
    
    # Format blocks
    formatted_blocks = []
    for block in blocks:
        block_type = block.get("type")
        
        # Default structure for a formatted block
        formatted_block = {
            "id": block.get("id"),
            "type": block_type,
            "text": ""
        }
        
        # Extract text based on block type
        if block_type in ["paragraph", "heading_1", "heading_2", "heading_3", 
                         "bulleted_list_item", "numbered_list_item"]:
            # These block types have a similar structure
            rich_text = block.get(block_type, {}).get("rich_text", [])
            formatted_block["text"] = "".join([text.get("plain_text", "") for text in rich_text])
        
        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            formatted_block["text"] = "".join([text.get("plain_text", "") for text in rich_text])
            formatted_block["language"] = block.get("code", {}).get("language")
        
        # Add more block types as needed
        
        formatted_blocks.append(formatted_block)
    
    return {
        "id": page.get("id"),
        "created_time": page.get("created_time"),
        "last_edited_time": page.get("last_edited_time"),
        "title": title,
        "content": formatted_blocks
    }

In [11]:
result = await read_notion_page()

Reading Notion page with ID: 160ab8e5212680239210c6501484cd90
Page title: Interesting papers
Found 100 content blocks
Full content saved to notion_page_content.json

Preview of content:

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
Here's how RAG works:
Document Processing: Your documents are processed and converted into embeddings (vector representati
Storage: These embeddings are stored in a vector database or similar storage system
